# Generate Slippy Map Boxes 

More information can be found at: https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames

In [1]:
# Use the black code formatter
%load_ext lab_black

In [2]:
import math
import pathlib
from itertools import product, repeat

import geopandas as gpd
import numpy as np
import shapely.geometry as sgeom


def deg2num(lon_deg: float, lat_deg: float, zoom: int) -> tuple:
    """Derive tile number for point in WGS 84 given certain zoom level.

    Args:
        lon_deg (float): Longitude as float in WGS 84.
        lat_deg (float):Latitude as float in WGS 84.
        zoom (int): Zoom following OSM slippy tile names conventions.

    Returns:
        tuple: tile number (xtile, ytile) following OSM slippy tile names conventions.
    """

    lat_rad = math.radians(lat_deg)
    n = 2.0**zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
    return (xtile, ytile)


def num2deg(xtile: int, ytile: int, zoom: int) -> tuple:
    """Tile number (xtile, ytile) to WGS 84 point (longitude, latitude) of northwest tile corner.

    Args:
        xtile (int): xtile number as int.
        ytile (int): ytile number as int int.
        zoom (int): zoom level following OSM slippy map names convention.

    Returns:
        tuple: point (longitude, latitude) in WGS 84.
    """
    n = 2.0**zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lon_deg, lat_deg)


def unique_items(tile_numbers: list) -> list:
    """Filter unique tuples from list of tuples.

    Args:
        tile_numbers (list): List of tuples.

    Returns:
        list: List of tuples.
    """
    seen = set()
    result = []
    for item in tile_numbers:
        if item not in seen:
            seen.add(item)
            result.append(item)
    return result


def coords2num(lons: list, lats: list, zoom: int) -> list:
    """List of longitude / latitude coordinates to OpenStreetMap tile names given certain zoom level.

    Args:
        lons (list): List of longitude as floats in WGS 84.
        lats (list): List of latitudes as floats in WGS 84.
        zoom (int): Zoom level following OSM slippy tile names.

    Returns:
        list: List of tuples with tile numbers (xtile, ytile).
    """
    tile_numbers = list(map(deg2num, lons, lats, repeat(zoom)))

    return unique_items(tile_numbers)


def zoom2num(zoom: int) -> list:
    """Generate tile numbers for zoom level.

    Args:
        zoom (int): zoom level in accordance with OpenStreetMap slippy tile names.

    Returns:
        list: list with tuples of tile numbers.
    """
    # tiles (zoom level) = 2**zoom * 2**zoom
    ntiles = 2**zoom
    tile_arange = np.arange(ntiles)
    return list(product(tile_arange, tile_arange))


def tile2box(xtile: int, ytile: int, zoom: int) -> dict:
    """Generate box for tile (xtile, ytile) at specified zoom level following OpenStreetMap
    slippy tile names conventions.

    Args:
        xtile int: xtile coordinate
        ytile int: ytile coordinate (int)
        zoom int: zoom level

    Returns:
        dict: with box name (zoom + xtile + ytile) as key and box (shapely.geometry.Polygon) as value.
    """
    nw = num2deg(xtile, ytile, zoom)
    ne = num2deg(xtile + 1, ytile, zoom)
    sw = num2deg(xtile, ytile + 1, zoom)
    se = num2deg(xtile + 1, ytile + 1, zoom)
    boxname = f"z{zoom}x{xtile}y{ytile}"
    return {boxname: sgeom.Polygon([nw, ne, sw, se]).envelope}

### Set configurations 

In [3]:
# Set zoom level and generate xtiles and ytiles
zoom = 8
tile_numbers = zoom2num(zoom)

# Convert the tile coordinates (xtile, ytile) to boxes by deriving the corners.
# Note, tile2box function is wrapped within lambda function to unpack tuples
# (xtile, ytile) in map
boxes = list(map(lambda xytile: tile2box(*xytile, zoom=zoom), tile_numbers))

# Result from previous function is a list of dictionaries. Here, merge into one dict.
boxes = {k: v for d in boxes for k, v in d.items()}

# Load into GeoPandas so that the results, for example, can be exported to GeoJSON
boxes = gpd.GeoDataFrame(boxes.items(), columns=["box_id", "geometry"], crs="EPSG:4326")

### Explore results

In [4]:
boxes

,box_id,geometry
0,z8x0y0,"POLYGON ((-180.00000 84.92832, -178.59375 84.9..."
1,z8x0y1,"POLYGON ((-180.00000 84.80247, -178.59375 84.8..."
2,z8x0y2,"POLYGON ((-180.00000 84.67351, -178.59375 84.6..."
3,z8x0y3,"POLYGON ((-180.00000 84.54136, -178.59375 84.5..."
4,z8x0y4,"POLYGON ((-180.00000 84.40594, -178.59375 84.4..."
...,...,...
65531,z8x255y251,"POLYGON ((178.59375 -84.54136, 180.00000 -84.5..."
65532,z8x255y252,"POLYGON ((178.59375 -84.67351, 180.00000 -84.6..."
65533,z8x255y253,"POLYGON ((178.59375 -84.80247, 180.00000 -84.8..."
65534,z8x255y254,"POLYGON ((178.59375 -84.92832, 180.00000 -84.9..."


In [5]:
boxes.sample(100).explore()

### Optional: export results

In [ ]:
# export_dir = pathlib.Path("ExamplePath")
# fpath = export_dir.joinpath(f"osm_slippy_boxes_z{zoom}.geojson"),
# boxes.to_file(
#     fpath
#     driver="GeoJSON",
# )